In [57]:
import os
import sys
from importnb import Notebook
from typing import Optional, Union, List, Callable, Any
from io import BytesIO
import struct
import numpy as np
import pandas as pd
from itertools import chain
from enum import Enum
import re

PKG_ROOT = os.path.dirname(os.path.realpath(os.getcwd()))
if not PKG_ROOT in sys.path:
    sys.path.append(PKG_ROOT)

In [58]:
with Notebook():
    from src.dtypes2 import *

In [59]:
OptimValues =   {'Gold': 1000000
                ,'Str': 100
                ,'Dex': 100
                ,'Int': 100
                ,'Wis': 100
                ,'Con': 100
                ,'Cha': 100
                #,'NumAttacks':5
                #,'BaseAttackBonus': 100
                ,'Rank': 50}

Feats = pd.read_csv(os.path.join(PKG_ROOT, 'resources', 'feats.csv'), index_col=0)
Feats = Feats[Feats['Selected']==1]['FeatIndex'].tolist()

Spells = pd.read_csv(os.path.join(PKG_ROOT, 'resources', 'spells.csv'), index_col=0)
Spells = Spells.groupby('SpellLevel')['SpellCode'].apply(lambda x:x.values)

Items = pd.read_csv(os.path.join(PKG_ROOT, 'resources', 'items.csv'), index_col=0)
Items = Enum('ITEMS', Items[['Item', 'ItemCode']].values)

In [60]:
pd.set_option('display.max_rows', 200)

In [61]:
class Player:
 
    HeaderPattern = '4s4s12I'
    StructPattern = '3I'
    LabelPattern = '16s'
    FieldPattern = '2I'
    DwordPattern = 'I'
    HeaderSize = struct.calcsize(HeaderPattern)
    StructSize = struct.calcsize(StructPattern)
    LabelSize = struct.calcsize(LabelPattern)
    FieldSize = struct.calcsize(FieldPattern)
    DwordSize = struct.calcsize(DwordPattern)
    
    def __init__(self, filename:str):
        self.source = os.path.join(PKG_ROOT, 'saves/original', filename)
        self.save = os.path.join (PKG_ROOT, 'saves/modified', filename)
        with open(self.source, 'rb') as s:
            self.buffer = s.read()
        self.structs = []
        self.fields = []
        self.labels = []
        self.parse_header()
        self.unpack()
        
    def parse_header(self):
        (self.FileType
         ,self.Version
         ,self.structOffset
         ,self.structCount
         ,self.fieldOffset
         ,self.fieldCount
         ,self.labelOffset
         ,self.labelCount
         ,self.dataOffset
         ,self.dataSize
         ,self.indiceOffset
         ,self.indiceSize
         ,self.listOffset
         ,self.listSize) = struct.unpack(self.HeaderPattern, self.buffer[:self.HeaderSize])
        self.source = {'struct': BytesIO(self.buffer[self.structOffset: self.fieldOffset])
                       ,'field': BytesIO(self.buffer[self.fieldOffset: self.labelOffset])
                       ,'label': BytesIO(self.buffer[self.labelOffset: self.dataOffset])
                       ,'data': BytesIO(self.buffer[self.dataOffset: self.indiceOffset])
                       ,'indice': BytesIO(self.buffer[self.indiceOffset: self.listOffset])
                       ,'list': BytesIO(self.buffer[self.listOffset:])
                       }
        self.target = {k: b'' for k in self.source.keys()}
        
    def headersize(self):
        for name in ('structCount', 'fieldCount', 'labelCount', 'dataSize', 'indiceSize', 'listSize'):
            print(f'{name}: {getattr(self, name)}')

    def unpack(self):
        self.labels = [struct.unpack(self.LabelPattern, self.source['label'].read(self.LabelSize))[0].decode('latin').rstrip('\x00') for _ in range(self.labelCount)]
        i = -1
        self.structs = [NWStruct(i:=i+1).unpack(self.source) for _ in range(self.structCount)]
        i = -1
        self.fields = [NWField(i:=i+1).unpack(self.source, self.structs, self.labels) for _ in range(self.fieldCount)]
        [_.bind_fields(self.fields) for _ in self.structs]

        
    def pack(self, save:Optional[str]=None):
        self.target = {k: b'' for k in self.source.keys()}
        labels = list()
        pack = self.structs[0].pack(self.target, labels)
        self.target['struct'] = pack + self.target['struct']
        self.target['label'] = b''.join([struct.pack(self.LabelPattern, _.encode('latin')) for _ in labels])
        header = struct.pack("4s4s", self.FileType, self.Version)
        content = b''
        offset = struct.calcsize(self.HeaderPattern)
        for k, v in self.target.items():
            if k == 'struct':
                length = len(self.target[k]) // self.StructSize
            elif k == 'field':
                length = len(self.target[k]) // 12
            elif k == 'label':
                length = len(self.target[k]) // self.LabelSize
            else:
                length = len(v)
            header += struct.pack("2I", offset + len(content), length)
            content += v
        content = header + content
        if save is None:
            save = self.save
        with open(save, 'wb') as t:
            t.write(content)
            
            
    def field_summary(self):
        return pd.DataFrame([_.summary() for _ in self.fields])
    
    
    def change_simple_value(self, label:str, newValue:Any, df:Optional[pd.DataFrame]=None):
        if df is None:
            df = self.field_summary()
        match = df[df['Label'] == label].copy()
        match['newValue'] = newValue # flexible for single value or value array
        if not df.empty:
            for f, v in match[['Field', 'newValue']].values:
                f.set_value(v)
                
    
    def add_similar_to_list(self, label:str, newValues:List, allowDuplicates:bool=False, deleteExisting:bool=False, df:Optional[pd.DataFrame]=None):
        '''
        Add more structs to the field with list content, newValues is a list, each element contains a set of values (can be a single value) to construct the new struct
        when constructed, the set of value is filled consecutivelu into each field of this struct
        '''
        if df is None:
            df = self.field_summary()
        match = df[df['Label'] == label]
        if not df.empty:
            for field in match['Field']:
                self.extend_list(field, newValues, allowDuplicates=allowDuplicates, deleteExisting=deleteExisting)
                
    
    def extend_list(self, field:NWField, newValues:List, allowDuplicates:bool=False, deleteExisting:bool=False):
        '''
        Make same structs but using new values suppied and add them to the list field
        Using the first struct in the list as template
        '''
        structIndex = len(self.structs) - 1
        structs = field.content.value
        if len(newValues) == 0:
            return
        if not isinstance(newValues[0], (list, dict, tuple)):
            newValues = [[_] for _ in newValues]  #consider newValues is a list of value for structs with one field, or list of value sets (for structs with multiple fields)
        existValues = [[f.content.value for f in s.fields] for s in structs] 
        if allowDuplicates == False:
            newValues = [_ for _ in newValues if not _ in existValues]
        sTemplate = structs[0]
        newStructs = [sTemplate.like(structIndex:=structIndex+1, v) for v in  newValues]
        self.structs.extend(newStructs)
        if deleteExisting == True:
            field.content.value = newStructs
        else:
            field.content.value = field.content.value + newStructs
        
    
    def optimize_feat(self, newFeats:List, label:str='FeatList', df:Optional[pd.DataFrame]=None):
        '''
        Some items like feat were added in batches, the field in struct#0 contains all known feats, and the rest structs contains feats added at different levels. Solution is add everything to the field in struct0, add everything different than those in 3rd to last structs to struct#2
        '''
        if df is None:
            df = self.field_summary()
        match = df[df['Label'] == label].sort_values('StructIndex')
        lateValues = list(chain(*match.iloc[2:]['Value'])) #laterly added feats
        self.extend_list(match.iloc[0]['Field'], newFeats)
        self.extend_list(match.iloc[1]['Field'], [_ for _ in newFeats if not _ in lateValues])
        
           
    def assign_spell(self, struct:NWStruct, level:Optional[int]=None):
        if level is None:
            level = Spells.size - 1
        structIndex = len(self.structs) - 1
        for level, spells in list(Spells.items())[:level + 1]:
            fLabel = 'KnownList' + str(level)
            newStructs = [NWStruct.new(3
                                       ,structIndex:=structIndex + 1
                                       ,[NWField.new(None
                                                     ,2
                                                     ,'Spell'
                                                     ,sCode)] 
                                       ) for sCode in spells]
            self.structs.extend(newStructs)
            struct.fields.append(NWField.new(None, 15, fLabel, newStructs))
            
            
    def assign_class(self
                     ,classDict:List
                     ,df:Optional[pd.DataFrame]=None):
        if df is None:
            df = self.field_summary()
        classStructs = df[df['Label']=='ClassList'].iloc[0]['Field']
        self.extend_list(classStructs, classDict, deleteExisting=True)
        
        
    def get_top_field(self
                  ,label:str
                  ,df:Optional[pd.DataFrame]=None):
        if df is None:
            df = self.field_summary()
        df = df[df['Label'] == label]
        return df.iloc[0]['Field']

                
    def optimize(self):
        df = self.field_summary()
        self.optimize_feat(Feats)
        self.add_similar_to_list('SkillList', range(28), df=df)
        classStructs = df[df['Label']=='ClassList'].iloc[0]['Field'].content.value
        self.assign_class([{'Class': 6, 'ClassLevel': 1}
                          ,{'Class': 10, 'ClassLevel': 22}
                          ,{'Class': 3, 'ClassLevel': 17}]
                          ,df=df)
        classStructs = df[df['Label']=='ClassList'].iloc[0]['Field'].content.value
        self.assign_spell(classStructs[1])
        for k, v in OptimValues.items():
            self.change_simple_value(k, v)
        self.pack()
                    
        

In [62]:
s1 = Player("amithrawiengalu1.bic")
s2 = Player("alarisewillin1.bic")

In [63]:
s2.optimize()

In [64]:
for label in ['ItemList', 'Equip_ItemList', 'QBList']:
    s2.get_top_field(label).content.value = s1.get_top_field(label).content.value 

In [66]:
s2.get_top_field('ClassList').content.value = [s2.get_top_field('ClassList').content.value[0]] + s1.get_top_field('ClassList').content.value[1:]

In [ ]:
s2.pack()